# core

In [ ]:
#| default_exp core

In [ ]:
from fastcore.all import *
from yt_dlp import YoutubeDL
import json

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
playlist_url = 'https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU'

In [ ]:
#| export
class YTPlaylist:
    def __init__(self, data:dict): store_attr()

    def __repr__(self):
        flds = ['webpage_url', 'title', 'channel', 'playlist_count']
        sig = ', '.join(f'{o}={self.data[o]!r}' for o in flds)
        return f'YTPlaylist({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTPlaylist, url: str)->YTPlaylist:
    with YoutubeDL({'flat_playlist': True, 'extract_flat': True, 'quiet': True}) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTPlaylist(data)

In [ ]:
playlist = YTPlaylist.from_url(playlist_url)
playlist

YTPlaylist(webpage_url='https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU', title='Practical Deep Learning for Coders', channel='Jeremy Howard', playlist_count=8)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()